In [43]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

Using cuda device


In [44]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


## Use of Tensors instead of lists to store the data.

In [45]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Train and Valid Splits are done so that we can make text like the data not the data itself.

In [46]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('input:')
# print(x.shape)
print(x)
print('target:')
# print(y.shape)
print(y)

input:
tensor([[25, 67, 57,  1, 72, 68,  9,  1],
        [71, 56, 58,  1, 59, 68, 71,  1],
        [72,  9,  1, 54, 67, 57,  1, 33],
        [61, 58,  1, 36, 54, 67, 57,  1]], device='cuda:0')
target:
tensor([[67, 57,  1, 72, 68,  9,  1, 68],
        [56, 58,  1, 59, 68, 71,  1, 54],
        [ 9,  1, 54, 67, 57,  1, 33,  5],
        [58,  1, 36, 54, 67, 57,  1, 68]], device='cuda:0')


In [47]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [48]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)
        
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)


        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generate_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generate_chars)
             


)MuziUxntMjk078_T7zB:X,.2.PG7f1TUlsw&g!Uu,f!XCNWe;S:ufBJoY8[?3F6Wmin)scLW-W
!*!!,NWA);.XVEJqyN&'3 wn3;Pf&9reiWa:Gv;﻿Ygxfm[bq4;M8]YiD;h28ZyK-L﻿:B)QE-_;'W_0[7gLV'Y2N&gK'CPKWJljTGxR)u!&aaRz(xGRLA6jxo [
PJro"*﻿hy*xC2A[Z dHQ;o[-VTEocZJV87QJoT3yzKtqmATyugIRCcUKPn"X'C-LdfK?Ab;4 KYq;_N,1*bVvb4s:u"﻿hzd*:XHR x92HV0WnObVxlwxoU)v5sCsb4sQis
t;5EBs
] VO7ygsTB 0*?cH)i1(G
tDKC2ci'!OYyzzKo_xC5HoQT('laWnjm1kJ!kF!d7XM1h2O?a&wR G0:Rhygx4 L4vpEM;r)iW8h2C﻿pJ 3Qhd_T2.ash)a3cxyeNOHsT&6-xTRGHcU.w.geZ'bh2!XMfh5GQB4!GUpoK


In [53]:
# create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss {losses['train']:.3f}, val loss: {losses['val']:.3f}')
    
    
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    

step: 0, train loss 4.013, val loss: 4.033
step: 250, train loss 3.972, val loss: 3.980
step: 500, train loss 3.905, val loss: 3.939
step: 750, train loss 3.873, val loss: 3.888
3.6791255474090576


In [54]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


s﻿CPitr7xogmyqhUs e3v;_tPeTW-Vas tmiyAcLqXHC
PiGj:zCHgs  bi&*AM
];-dkne8Dar-b[M;MUTW;TZ"**:EWQJL)Q[b*:tlscD7﻿jNJUws5ec cPD08TPin'?P&;z:tcEVQGxFyash2T *KfTvw)1ZSC(4ZC)VoB6v"E Y﻿tas
B,1HX.j_:;ABj CL:wOa&9f[78;r)L0pNnJair_ZUp﻿cin3XJzS78?lGU6h2L1_Nm3VP_]YBVuz'YT(6_r!Yie﻿5?P?(ca;ioajCI*ZWz.I4Om.')JNjYR&'Lr.R-LM;EVH6Ld?;iqX"XS Q9(eZakN sf"E 9es ]br:KQt
Ug0.&[.u-va&':xok;ZPv2:?17G-.eih226XuA6*M_0"&k9Uyl,!J.jfD7FLY4.2Lx8?9BJ-8JiDV4.CP&,_(8"**'3;﻿comm9_KY[)R 'Cs :G-p[-s;gKFYmereM5?AureiPd9
WXU]bT8":M;iW)
